In [ ]:
pyodbc

In [ ]:
#pip install pyodbc

In [1]:
import pyodbc

## SQL Server Connection

In [2]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try: 
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [ ]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'master'#yeni bir db oluşturacağız
user = 'sa' 
password = '------'
conn = create_server_connection(driver, server, database, user, password)

In [11]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 13 for SQL Server',
 'ODBC Driver 17 for SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)']

### windows authentication

In [2]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [3]:
conn = pyodbc.connect(conn_string);

In [4]:
crs = conn.cursor()

In [5]:
conn.autocommit = True

In [ ]:
#conn.commit()

In [8]:
create_database_query = 'Create database test_1'

In [6]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try: 
        crs.execute(create_database_query)
        print("Database is Created Succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [9]:
create_database(conn, create_database_query)

Database is Created Succesfully


In [6]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [7]:
query = 'USE Test_1'
execute_query(conn,query)

Query Succeessful!


## Create Table 

In [13]:
query = 'CREATE TABLE TestA (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

execute_query(conn, query)

Query Succeessful!


## Insert

In [8]:
crs= conn.cursor()
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")
#conn.commit()

In [9]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bruce', 'Lee')

In [10]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

In [11]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

## Delete 

In [12]:
crs.execute("DELETE FROM TestA WHERE FirstName = 'Jerry'")

In [13]:
crs.execute("DELETE FROM TestA WHERE FirstName = ?" ,'Stefan')

## Rowcount 

In [21]:
crs.rowcount

1

In [22]:
print(crs.rowcount,"row(s) deleted")

1 row(s) deleted


## read data

### fetchone()

In [23]:
crs.execute('SELECT FirstName, LastName FROM TestA')
row = crs.fetchone() #tablodaki ilk kaydı getirir
row

('Bob', 'Marley')

In [24]:
row = crs.fetchone() #tablodaki ilk kaydı getirir
row

('Bruce', 'Lee')

In [26]:
row.FirstName

'Bruce'

In [27]:
row.LastName

'Lee'

## fetchall()

In [14]:
crs.execute('SELECT ID, FirstName, LastName FROM TestA')
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William'),
 (9, 'Bob', 'Marley'),
 (10, 'Bruce', 'Lee'),
 (11, 'Tom', 'Hanks'),
 (12, 'Bart', 'Simpson'),
 (13, 'Tom', 'Cat'),
 (15, 'Owen', 'William')]

In [29]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Tom Cat
7 Owen William


In [30]:
crs.execute("""SELECT ID, FirstName, LastName 
                FROM TestA""")
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [31]:
type(rows)

list

In [32]:
crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestA
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows

[(7, 'Owen', 'William')]

## Fetchmany()

In [33]:
crs.execute("""SELECT ID, FirstName, LastName 
               FROM TestA""")
rows = crs.fetchmany(2)
rows

[(1, 'Bob', 'Marley'), (2, 'Bruce', 'Lee')]

## Fetchval()

In [34]:
crs.execute("SELECT MAX(ID) FROM TestA").fetchval()

7

In [36]:
crs.execute("SELECT FirstName FROM TestA").fetchval()

'Bob'

## Pandas Methods

In [15]:
import pandas as pd

In [16]:
def read_query (conn, query): #connection nesnesi yukarıda tanımlanmıştı.
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [17]:
query = """SELECT ID, FirstName, LastName 
               FROM TestA"""




In [18]:
result = read_query (conn, query)

In [19]:
result

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William'),
 (9, 'Bob', 'Marley'),
 (10, 'Bruce', 'Lee'),
 (11, 'Tom', 'Hanks'),
 (12, 'Bart', 'Simpson'),
 (13, 'Tom', 'Cat'),
 (15, 'Owen', 'William')]

In [20]:
df= pd.DataFrame(result)

In [21]:
df

,0
0,"[1, Bob, Marley]"
1,"[2, Bruce, Lee]"
2,"[3, Tom, Hanks]"
3,"[4, Bart, Simpson]"
4,"[5, Tom, Cat]"
5,"[7, Owen, William]"
6,"[9, Bob, Marley]"
7,"[10, Bruce, Lee]"
8,"[11, Tom, Hanks]"
9,"[12, Bart, Simpson]"


In [22]:
from_db = []

for result in result:
    result = list(result)
    from_db.append(result)

In [23]:
from_db

[[1, 'Bob', 'Marley'],
 [2, 'Bruce', 'Lee'],
 [3, 'Tom', 'Hanks'],
 [4, 'Bart', 'Simpson'],
 [5, 'Tom', 'Cat'],
 [7, 'Owen', 'William'],
 [9, 'Bob', 'Marley'],
 [10, 'Bruce', 'Lee'],
 [11, 'Tom', 'Hanks'],
 [12, 'Bart', 'Simpson'],
 [13, 'Tom', 'Cat'],
 [15, 'Owen', 'William']]

In [24]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(from_db, columns = columns)
df

,ID,FirstName,LastName
0,1,Bob,Marley
1,2,Bruce,Lee
2,3,Tom,Hanks
3,4,Bart,Simpson
4,5,Tom,Cat
5,7,Owen,William
6,9,Bob,Marley
7,10,Bruce,Lee
8,11,Tom,Hanks
9,12,Bart,Simpson


In [25]:
query = 'USE BikeStores'
execute_query(conn,query)

Query Succeessful!


In [26]:
crs.execute("SELECT * FROM production.brands").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Heller'),
 (4, 'Pure Cycles'),
 (5, 'Ritchey'),
 (6, 'Strider'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [27]:
pd.read_sql("SELECT * FROM production.brands", con = conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [28]:
df_prod = pd.read_sql("SELECT * FROM production.products", con = conn)
df_prod.head(10)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,8,6,2016,469.99
6,7,Trek Slash 8 27.5 - 2016,9,6,2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,9,6,2016,1799.99
8,9,Trek Conduit+ - 2016,9,5,2016,2999.99
9,10,Surly Straggler - 2016,8,4,2016,1549.00


## DF to Databese

In [30]:
import sqlalchemy
import urllib

In [ ]:
crs.execute('USE Test_1')

In [33]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'Test_1'#yeni bir db oluşturacağız
user = 'sa' 
password = '---'

In [34]:
params = 'DRIVER='+driver + ';SERVER=' + server +';PORT=1433;DATABASE='+ database + ';UID=' + user + ';PWD=' + password 

In [35]:
db_params = urllib.parse.quote_plus (params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [36]:
df_prod = pd.read_sql("SELECT * FROM production.products where product_id <10", con = conn)
df_prod.head(10)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,8,6,2016,469.99
6,7,Trek Slash 8 27.5 - 2016,9,6,2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,9,6,2016,1799.99
8,9,Trek Conduit+ - 2016,9,5,2016,2999.99


In [37]:
df_prod.to_sql("products_bikes", engine, index = False, if_exists = "append", schema = "dbo")